In [1]:
print("Hello World!")

Hello World!


In [12]:
class PositiveValue:
    def __get__(self, instance, owner):
        print (f"__get__ has been called")
        return instance.__dict__.get(self.name, 0)
    
    def __set__(self, instance, value):
        print (f"__set__ has been called")
        if value < 0:
            raise ValueError("Value must be positive")
        instance.__dict__[self.name] = value
    
    def __set_name__(self, instance, name):
        print (f"__set_name__ has been called")
        self.name = name

class Account:
    def __init__(self):
        print(f"{self.__class__}.__init__ has been called")

    balance = PositiveValue()

a = Account()
a.balance = 100
print(a.balance)

__set_name__ has been called
<class '__main__.Account'>.__init__ has been called
__set__ has been called
__get__ has been called
100


In [13]:
class DatabaseField:
    def __get__(self, instance, owner):
        return f'Fetching the database {self.name}'

    def __set_name__(self, owner, name):
        self.name = name

class User:
    name = DatabaseField()
    email = DatabaseField()


someuser = User()
print(someuser.name)
print(someuser.email)

Fetching the database name
Fetching the database email


In [15]:
from datetime import datetime

class TimeUTC:
    def __get__(self, instance, owner):
        return datetime.utcnow().isoformat()
    

class Logger:
    current_time = TimeUTC()

l = Logger()
l.current_time

C:\Users\achakravarti\AppData\Local\Temp\ipykernel_12656\2979866351.py:5: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().isoformat()


'2024-12-15T06:09:22.497791'

In [16]:
from random import choice, seed
seed(0)


class Deck:
    @property
    def suit(self):
        return choice(('Spade', 'Heart', 'Diamond', 'Club'))
    
    @property
    def card(self):
        return choice(tuple('23456789JQKA') + ('10',))
    

d = Deck()

for _ in range(10):
    print(d.card, d.suit)

8 Club
2 Diamond
J Club
8 Diamond
9 Diamond
Q Heart
J Heart
6 Heart
10 Spade
Q Diamond


In [22]:
seed(0)

class randomChoice:
    def __init__(self, choices):
        self.choices = choices

    def __get__(self, instance, owner): 
        return choice(self.choices)
    
class Deck:
    suit = randomChoice(('Spade', 'Heart', 'Diamond', 'Club'))
    card = randomChoice(tuple('23456789JQKA') + ('10',))

d = Deck()

for _ in range(10):
    print(d.card, d.suit)

8 Club
2 Diamond
J Club
8 Diamond
9 Diamond
Q Heart
J Heart
6 Heart
10 Spade
Q Diamond


In [29]:
from datetime import datetime

class TimeUTC:
    def __get__(self, instance, owner):
        print (f'__get__ is called, self = {self}, instance = {instance}, owner = {owner}')
        return datetime.utcnow().isoformat()
    
class Logger1:
    current_time = TimeUTC()
    print(hex(id(current_time)))

class Logger2:
    current_time = TimeUTC()

obj1 = Logger1()
obj1.current_time
print(hex(id(obj1)))


0x1c1f310e7b0
__get__ is called, self = <__main__.TimeUTC object at 0x000001C1F310E7B0>, instance = <__main__.Logger1 object at 0x000001C1F039D700>, owner = <class '__main__.Logger1'>
0x1c1f039d700


C:\Users\achakravarti\AppData\Local\Temp\ipykernel_12656\1397531809.py:6: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().isoformat()


In [39]:
class IntegerValue:
    def __init__ (self):
        print(hex(id(self)))
        self.values = {}
    
    def __set__(self, instance, value):
        self.values[instance] = value
    
    def __get__(self, instance, owner):
        if instance is None:
            return self
        else:
            return self.values.get(instance)
        
class Point2D:
    x = IntegerValue()
    print(f' x = {hex(id(x))}')
    y = IntegerValue()
    print(f'y = {hex(id(y))}')

p1 = Point2D()
p2 = Point2D()

p1.x = 10
p1.y = 20

p2.x = 100
p2.y = 200

print('\n\n')
print(Point2D.__dict__)
print('\n\n')
print(p1.x, p1.y, p2.x, p2.y)

0x1c1f2ff63c0
 x = 0x1c1f2ff63c0
0x1c1f3137f50
y = 0x1c1f3137f50



{'__module__': '__main__', 'x': <__main__.IntegerValue object at 0x000001C1F2FF63C0>, 'y': <__main__.IntegerValue object at 0x000001C1F3137F50>, '__dict__': <attribute '__dict__' of 'Point2D' objects>, '__weakref__': <attribute '__weakref__' of 'Point2D' objects>, '__doc__': None}



10 20 100 200


In [40]:
Point2D.x.values

{<__main__.Point2D at 0x1c1f3134bf0>: 10,
 <__main__.Point2D at 0x1c1f31351c0>: 100}

# Strong and Weak References

In [41]:
import ctypes

def ref_count(address):
    return ctypes.c_long.from_address(address).value

In [42]:
import weakref


In [61]:
class Person:
    def __init__(self, name):
        self.name = name
        
    def __repr__(self):
        return f'Person(name={self.name})'
    
p1 = Person('Guido')

p1_id = id(p1)

print(ref_count(p1_id))
print(hex(id(p1)))

1
0x1c1f3134e30


In [62]:
p2 = p1
ref_count(p1_id)

2

In [63]:
weak1 = weakref.ref(p1)
ref_count(p1_id)
print(weak1)
hex(id(weak1))

<weakref at 0x000001C1F3641B70; to 'Person' at 0x000001C1F3134E30>


'0x1c1f3641b70'

In [64]:
something = weak1()
ref_count(p1_id)

3

In [66]:
weakref.getweakrefcount(p1)

1

In [67]:
del p1
del p2
del something

In [72]:
ref_count(p1_id)

100

In [74]:
obj = weak1()

obj is None

True

In [77]:
p1 = Person('Guido')

d = weakref.WeakKeyDictionary()

print(ref_count(id(p1)))
weakref.getweakrefcount(p1)

1


0

In [ ]:
d[p1] = 'Guido'

In [78]:
import weakref

class IntegerValue:
    def __init__(self):
        self.values = weakref.WeakKeyDictionary()

    def __set__(self, instance, value):
        self.values[instance] = int(value)

    def __get__(self, instance, owner):
        return self.values.get(instance)

In [4]:
var = "aravind.daccord.com"
if '.' in var and '@' in var:
    print("hello")